## 配置运行环境

In [ ]:
# 重启
#!kill -9 -1
# 安装包
# !pip install packagename

In [3]:
# 挂载谷歌硬盘 从drive导入数据
from google.colab import drive
drive.mount('/content/drive')
# 创建数据文件夹
!mkdir /content/dataset
!mkdir .kaggle
# 复制文件到本地
# !cp /content/drive/My\ Drive/Dataset/filename /content/dataset/filename
# 切换当前目录
# %cd /content/dataset/
# 解压缩文件
# !unzip filename


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/dataset’: File exists
mkdir: cannot create directory ‘.kaggle’: File exists
/content/dataset
Archive:  sanion.zip
  inflating: sub.csv                 
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# 使用小数据集
# !cp /content/drive/My\ Drive/Colab_Notebooks/RS6/L12/santander-product-recommendation.zip /content/dataset/sanion.zip
# %cd /content/dataset/
# !unzip sanion.zip

In [7]:
# 从kaggle导入数据
!cp /content/drive/My\ Drive/Dataset/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
# !kaggle datasets list
!kaggle datasets list -s santander #搜索指定数据集
!kaggle datasets download -d marialuiza07/santander-product-recomendation -p /content/dataset/
%cd /content/dataset/
!unzip santander-product-recomendation.zip

Archive:  santander-product-recomendation.zip
  inflating: s-product-recomendation/sample_submission.csv  
  inflating: s-product-recomendation/test_ver2.csv  
  inflating: s-product-recomendation/train_ver2.csv  
  inflating: sample_submission.csv   
  inflating: test_ver2.csv           
  inflating: train_ver2.csv          


## 数据预处理

In [4]:
# 导入数据
import pandas as pd
import time
import numpy as np
# 全量数据集
# train = pd.read_csv('train_ver2.csv',dtype={"sexo":str, "ind_nuevo":str, "ult_fec_cli_1t":str, "indext":str})
# test = pd.read_csv("test_ver2.csv",dtype={"sexo":str, "ind_nuevo":str, "ult_fec_cli_1t":str, "indext":str})

# 小量数据集
train = pd.read_csv('train.csv',dtype={"sexo":str, "ind_nuevo":str, "ult_fec_cli_1t":str, "indext":str})
test = pd.read_csv("test.csv",dtype={"sexo":str, "ind_nuevo":str, "ult_fec_cli_1t":str, "indext":str})

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# 数据概览
print(train.shape)
print(test.shape)
print(train.columns)
print(test.columns)

(533770, 48)
(29162, 24)
Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'],
      dtype='object')
Index(['fecha_dato', 'ncodpers', 'ind_empleado', 

In [6]:
# 对24个label进行处理
targets = train.columns[24:]

# 查看label分布
for target in targets:
    print(target+":\n"+str(train[target].value_counts()))

ind_ahor_fin_ult1:
0    533722
1        48
Name: ind_ahor_fin_ult1, dtype: int64
ind_aval_fin_ult1:
0    533754
1        16
Name: ind_aval_fin_ult1, dtype: int64
ind_cco_fin_ult1:
1    351209
0    182561
Name: ind_cco_fin_ult1, dtype: int64
ind_cder_fin_ult1:
0    533494
1       276
Name: ind_cder_fin_ult1, dtype: int64
ind_cno_fin_ult1:
0    490450
1     43320
Name: ind_cno_fin_ult1, dtype: int64
ind_ctju_fin_ult1:
0    528637
1      5133
Name: ind_ctju_fin_ult1, dtype: int64
ind_ctma_fin_ult1:
0    528571
1      5199
Name: ind_ctma_fin_ult1, dtype: int64
ind_ctop_fin_ult1:
0    462330
1     71440
Name: ind_ctop_fin_ult1, dtype: int64
ind_ctpp_fin_ult1:
0    509806
1     23964
Name: ind_ctpp_fin_ult1, dtype: int64
ind_deco_fin_ult1:
0    532742
1      1028
Name: ind_deco_fin_ult1, dtype: int64
ind_deme_fin_ult1:
0    533005
1       765
Name: ind_deme_fin_ult1, dtype: int64
ind_dela_fin_ult1:
0    510820
1     22950
Name: ind_dela_fin_ult1, dtype: int64
ind_ecue_fin_ult1:
0    490232
1

In [7]:
# 查看label缺失值
for target in targets:
    print(target+":"+str(train[target].isnull().sum()))

ind_ahor_fin_ult1:0
ind_aval_fin_ult1:0
ind_cco_fin_ult1:0
ind_cder_fin_ult1:0
ind_cno_fin_ult1:0
ind_ctju_fin_ult1:0
ind_ctma_fin_ult1:0
ind_ctop_fin_ult1:0
ind_ctpp_fin_ult1:0
ind_deco_fin_ult1:0
ind_deme_fin_ult1:0
ind_dela_fin_ult1:0
ind_ecue_fin_ult1:0
ind_fond_fin_ult1:0
ind_hip_fin_ult1:0
ind_plan_fin_ult1:0
ind_pres_fin_ult1:0
ind_reca_fin_ult1:0
ind_tjcr_fin_ult1:0
ind_valo_fin_ult1:0
ind_viv_fin_ult1:0
ind_nomina_ult1:717
ind_nom_pens_ult1:717
ind_recibo_ult1:0


In [8]:
# 填充缺失值
train['ind_nom_pens_ult1'].fillna(0,inplace=True)
train['ind_nomina_ult1'].fillna(0,inplace=True)

In [9]:
# 统一label格式
for target in targets:
    train[target] = train[target].astype('int16')

In [10]:
# 查看特征的缺失值
train.isnull().sum()

fecha_dato                    0
ncodpers                      0
ind_empleado               1146
pais_residencia            1146
sexo                       1146
age                           0
fecha_alta                 1146
ind_nuevo                  1146
antiguedad                    0
indrel                     1146
ult_fec_cli_1t           532841
indrel_1mes                6150
tiprel_1mes                6150
indresi                    1146
indext                     1146
conyuemp                 533767
canal_entrada              7661
indfall                    1146
tipodom                    1146
cod_prov                   3769
nomprov                    3769
ind_actividad_cliente      1146
renta                    106821
segmento                   7782
ind_ahor_fin_ult1             0
ind_aval_fin_ult1             0
ind_cco_fin_ult1              0
ind_cder_fin_ult1             0
ind_cno_fin_ult1              0
ind_ctju_fin_ult1             0
ind_ctma_fin_ult1             0
ind_ctop

In [11]:
test.isnull().sum()

fecha_dato                   0
ncodpers                     0
ind_empleado                 0
pais_residencia              0
sexo                         0
age                          0
fecha_alta                   0
ind_nuevo                    0
antiguedad                   0
indrel                       0
ult_fec_cli_1t           29159
indrel_1mes                  0
tiprel_1mes                  0
indresi                      0
indext                       0
conyuemp                 29162
canal_entrada                2
indfall                      0
tipodom                      0
cod_prov                   139
nomprov                    139
ind_actividad_cliente        0
renta                     6757
segmento                     2
dtype: int64

In [12]:
# 两列特征缺失值太多，故删除
del train['ult_fec_cli_1t']
del test['ult_fec_cli_1t']
del train['conyuemp']
del test['conyuemp']

In [13]:
# 处理‘fecha_dato’字段
print(train['fecha_dato'].value_counts())
print(test['fecha_dato'].value_counts())
# 转换时间格式
train['fecha_dato']=pd.to_datetime(train['fecha_dato'], format='%Y-%m-%d')
test['fecha_dato']=pd.to_datetime(test['fecha_dato'], format='%Y-%m-%d')
# 构建'year'年和'month'月特征
train['year'] = train['fecha_dato'].dt.year.astype('int16')
train['month'] = train['fecha_dato'].dt.month.astype('int16')
test['year'] = test['fecha_dato'].dt.year.astype('int16')
test['month'] = test['fecha_dato'].dt.month.astype('int16')
# 构建特征时间戳
temp = train['fecha_dato'][0].value

train['timestamp'] = [(int(i.value-temp))/1e11 for i in train['fecha_dato']]
test['timestamp'] = [(int(i.value-temp))/1e11 for i in test['fecha_dato']]

train['timestamp'] = train['timestamp'].astype('int64')
test['timestamp'] = test['timestamp'].astype('int64')

del train['fecha_dato']
del test['fecha_dato']

2016-04-28    39006
2016-03-28    38858
2016-02-28    38703
2016-01-28    38511
2015-12-28    38334
2015-11-28    38061
2015-10-28    37484
2015-09-28    36328
2015-08-28    35403
2015-07-28    34849
2015-06-28    26516
2015-05-28    26484
2015-04-28    26417
2015-03-28    26354
2015-02-28    26272
2015-01-28    26190
Name: fecha_dato, dtype: int64
2016-05-28    29162
Name: fecha_dato, dtype: int64


In [14]:
# 处理'ncodpers'字段
print(train['ncodpers'].value_counts())
print(test['ncodpers'].value_counts())
print(train['ncodpers'].isnull().sum())
print(test['ncodpers'].isnull().sum())
train['ncodpers'] = train['ncodpers'].astype('int64')
test['ncodpers'] = test['ncodpers'].astype('int64')

1058811    16
1248435    16
1278114    16
1052880    16
28892      16
           ..
1546226     1
1545963     1
1542908     1
1543440     1
1444901     1
Name: ncodpers, Length: 40000, dtype: int64
724990     1
1099634    1
1031997    1
1153550    1
1038683    1
          ..
181197     1
1131888    1
404432     1
80850      1
1052674    1
Name: ncodpers, Length: 29162, dtype: int64
0
0


In [15]:
# 处理'ind_empleado'字段
print(train['ind_empleado'].value_counts())
print(test['ind_empleado'].value_counts())
print(train['ind_empleado'].isnull().sum())
print(test['ind_empleado'].isnull().sum())
# 众数填充训练集
train['ind_empleado'].fillna('N',inplace=True)

N    532482
B        68
A        32
F        26
S        16
Name: ind_empleado, dtype: int64
N    29155
B        3
A        2
S        1
F        1
Name: ind_empleado, dtype: int64
1146
0


In [16]:
# 创建列表le存放分类特征，后面一块处理
le = []
le.append('ind_empleado')

In [17]:
# 处理'pais_residencia'字段
print(train['pais_residencia'].value_counts().head())
print(test['pais_residencia'].value_counts().head())
print(train['pais_residencia'].isnull().sum())
print(test['pais_residencia'].isnull().sum())
#众数填充训练集
train['pais_residencia'].fillna('ES',inplace=True)
le.append('pais_residencia')

ES    529970
DE       256
FR       210
AR       196
CO       192
Name: pais_residencia, dtype: int64
ES    29023
DE       16
CO       11
AR       11
FR       10
Name: pais_residencia, dtype: int64
1146
0


In [18]:
# 处理 'sexo' 字段
print(train['sexo'].value_counts())
print(test['sexo'].value_counts())
print(train['sexo'].isnull().sum())
print(test['sexo'].isnull().sum())
# 用前一条数据的值填充
train['sexo'].fillna(method='pad', inplace=True)
le.append('sexo')

V    291340
H    241284
Name: sexo, dtype: int64
V    15724
H    13438
Name: sexo, dtype: int64
1146
0


In [19]:
# 处理 'age' 字段
print(train['age'].value_counts())
print(test['age'].value_counts())
print(train['age'].isnull().sum())
print(test['age'].isnull().sum())
train["age"]   = pd.to_numeric(train["age"],errors='coerce')
test['age'] = test['age'].astype('int64')
#众数填充训练集
train['age'].fillna(23,inplace=True)

train['age'] = train['age'].astype('int64')
test['age'] = test['age'].astype('int64')

23     20061
22     19889
24     19398
21     19084
20     14233
       ...  
9          2
111        1
97         1
8          1
98         1
Name: age, Length: 309, dtype: int64
24     1786
23     1589
22     1522
21     1395
25     1339
       ... 
103       2
2         1
102       1
107       1
110       1
Name: age, Length: 107, dtype: int64
0
0


In [20]:
# 处理 'fecha_alta' 字段
print(train['fecha_alta'].value_counts())
print(test['fecha_alta'].value_counts())
print(train['fecha_alta'].isnull().sum())
print(test['fecha_alta'].isnull().sum())
# 用众数填充
train['fecha_alta'].fillna('2014-07-28',inplace=True)
#构建时间特征
train['fecha_alta']=pd.to_datetime(train['fecha_alta'], format='%Y-%m-%d')
test['fecha_alta']=pd.to_datetime(test['fecha_alta'], format='%Y-%m-%d')

train['fyear'] = train['fecha_alta'].dt.year.astype('int16')
train['fmonth'] = train['fecha_alta'].dt.month.astype('int16')
train['fday'] = train['fecha_alta'].dt.day.astype('int16')
test['fyear'] = test['fecha_alta'].dt.year.astype('int16')
test['fmonth'] = test['fecha_alta'].dt.month.astype('int16')
test['fday'] = test['fecha_alta'].dt.day.astype('int16')

train['ftimestamp'] = [(int(i.value))/1e11 for i in train['fecha_alta']]
test['ftimestamp'] = [(int(i.value))/1e11 for i in test['fecha_alta']]
train['ftimestamp'] = train['ftimestamp'].astype('int64')
test['ftimestamp'] = test['ftimestamp'].astype('int64')

del train['fecha_alta']
del test['fecha_alta']

2014-07-28    2137
2014-10-03    2110
2013-10-14    2072
2014-08-04    1910
2013-08-03    1373
              ... 
2016-04-17       3
2016-02-28       2
2006-02-04       2
2016-04-03       1
2016-04-24       1
Name: fecha_alta, Length: 5668, dtype: int64
2014-07-28    126
2014-10-03    112
2013-10-14    108
2014-08-04    107
2013-08-03     78
             ... 
2004-06-29      1
2011-06-24      1
2002-12-13      1
2005-04-06      1
2009-05-05      1
Name: fecha_alta, Length: 5279, dtype: int64
1146
0


In [21]:
# 处理 'ind_nuevo' 字段
print(train['ind_nuevo'].value_counts())
print(test['ind_nuevo'].value_counts())
print(train['ind_nuevo'].isnull().sum())
print(test['ind_nuevo'].isnull().sum())
train['ind_nuevo'].fillna(0.0,inplace=True)
train['ind_nuevo'] = train['ind_nuevo'].astype('float16')
test['ind_nuevo'] = test['ind_nuevo'].astype('float16')
train['ind_nuevo'] = train['ind_nuevo'].astype('int16')
test['ind_nuevo'] = test['ind_nuevo'].astype('int16')

0.0    499694
1.0     32930
Name: ind_nuevo, dtype: int64
0.0    28317
1.0      845
Name: ind_nuevo, dtype: int64
1146
0


In [22]:
# 处理 'antiguedad' 字段
print(train['antiguedad'].value_counts().head())
print(test['antiguedad'].value_counts().head())
print(train['antiguedad'].isnull().sum())
print(test['antiguedad'].isnull().sum())

train["antiguedad"]   = pd.to_numeric(train["antiguedad"], errors="coerce")
test['antiguedad'] = test['antiguedad'].astype('int32')
train['antiguedad'].fillna(0,inplace=True)
train['antiguedad'] = train['antiguedad'].astype('int32')

0     5490
1     5328
12    5314
3     5078
2     5069
Name: antiguedad, dtype: int64
31    811
7     759
8     749
10    742
43    657
Name: antiguedad, dtype: int64
0
0


In [23]:
# 处理 'indrel' 字段
print(train['indrel'].value_counts())
print(test['indrel'].value_counts())
print(train['indrel'].isnull().sum())
print(test['indrel'].isnull().sum())
train['indrel'].fillna(0,inplace=True)

train['indrel'] = train['indrel'].astype('int32')
test['indrel'] = test['indrel'].astype('int32')

1.0     531695
99.0       929
Name: indrel, dtype: int64
1.0     29159
99.0        3
Name: indrel, dtype: int64
1146
0


In [24]:
# 处理 'indrel_1mes' 字段
print(train['indrel_1mes'].value_counts())
print(test['indrel_1mes'].value_counts())
print(train['indrel_1mes'].isnull().sum())
print(test['indrel_1mes'].isnull().sum())

train["indrel_1mes"]   = pd.to_numeric(train["indrel_1mes"], errors="coerce")

train['indrel_1mes'].fillna(5,inplace=True)
test['indrel_1mes'].fillna(5,inplace=True)

train['indrel_1mes'] = train['indrel_1mes'].astype('int32')
test['indrel_1mes'] = test['indrel_1mes'].astype('int32')

1.0    260936
1.0    205021
1       61417
3.0        86
3.0        46
P          29
3          26
2.0        21
2          14
2.0        12
4.0         6
4           3
4.0         3
Name: indrel_1mes, dtype: int64
1.0    29160
3.0        2
Name: indrel_1mes, dtype: int64
6150
0


In [25]:
#  处理 'tiprel_1mes' 字段
print(train['tiprel_1mes'].value_counts())
print(test['tiprel_1mes'].value_counts())
print(train['tiprel_1mes'].isnull().sum())
print(test['tiprel_1mes'].isnull().sum())

train['tiprel_1mes'].fillna('I',inplace=True)

le.append('tiprel_1mes')

I    284020
A    243401
P       170
R        29
Name: tiprel_1mes, dtype: int64
A    15626
I    13534
P        2
Name: tiprel_1mes, dtype: int64
6150
0


In [26]:
# 处理 'indresi' 字段
print(train['indresi'].value_counts())
print(test['indresi'].value_counts())
print(train['indresi'].isnull().sum())
print(test['indresi'].isnull().sum())

train['indresi'].fillna('S',inplace=True)

le.append('indresi')

S    529970
N      2654
Name: indresi, dtype: int64
S    29023
N      139
Name: indresi, dtype: int64
1146
0


In [27]:
# 处理 'indext' 字段
print(train['indext'].value_counts())
print(test['indext'].value_counts())

print(train['indext'].isnull().sum())
print(test['indext'].isnull().sum())

train['indext'].fillna('N',inplace=True)

le.append('indext')

N    507155
S     25469
Name: indext, dtype: int64
N    27867
S     1295
Name: indext, dtype: int64
1146
0


In [28]:
# 处理 'canal_entrada' 字段
print(train['canal_entrada'].value_counts())
print(test['canal_entrada'].value_counts())
print(train['canal_entrada'].isnull().sum())
print(test['canal_entrada'].isnull().sum())

train['canal_entrada'].fillna('KHE',inplace=True)
test['canal_entrada'].fillna('KHE',inplace=True)

le.append('canal_entrada')

KHE    157115
KAT    131924
KFC    120673
KHQ     21496
KFA     15783
        ...  
KBJ        10
KCK        10
KCS        10
KEU        10
KDN        10
Name: canal_entrada, Length: 135, dtype: int64
KHE    8588
KAT    6689
KFC    5854
KHQ    2265
KHM     973
       ... 
KDS       1
KDZ       1
KDQ       1
KCP       1
KBB       1
Name: canal_entrada, Length: 127, dtype: int64
7661
2


In [29]:
# 处理 'indfall' 字段
print(train['indfall'].value_counts())
print(test['indfall'].value_counts())

print(train['indfall'].isnull().sum())
print(test['indfall'].isnull().sum())

train['indfall'].fillna('N',inplace=True)

le.append('indfall')

N    531311
S      1313
Name: indfall, dtype: int64
N    29109
S       53
Name: indfall, dtype: int64
1146
0


In [30]:
# 处理 'tipodom' 字段
print(train['tipodom'].value_counts())
print(test['tipodom'].value_counts())

print(train['tipodom'].isnull().sum())
print(test['tipodom'].isnull().sum())

#都是1，选择删除特征
del train['tipodom']
del test['tipodom']

1.0    532624
Name: tipodom, dtype: int64
1.0    29162
Name: tipodom, dtype: int64
1146
0


In [31]:
# 处理 'cod_prov' 字段
print(train['cod_prov'].value_counts().head())
print(test['cod_prov'].value_counts().head())
print(train['cod_prov'].isnull().sum())
print(test['cod_prov'].isnull().sum())

#众数填充
train['cod_prov'].fillna('28.0',inplace=True)
test['cod_prov'].fillna('28.0',inplace=True)

train['cod_prov'] = train['cod_prov'].astype('float64')
test['cod_prov'] = test['cod_prov'].astype('float64')
train['cod_prov'] = train['cod_prov'].astype('int64')
test['cod_prov'] = test['cod_prov'].astype('int64')

28.0    173162
8.0      50295
46.0     26182
41.0     24350
15.0     16850
Name: cod_prov, dtype: int64
28.0    8921
8.0     2548
46.0    1510
41.0    1391
15.0     989
Name: cod_prov, dtype: int64
3769
139


In [32]:
# 处理 'nomprov' 字段

print(train['nomprov'].value_counts().head())
print(test['nomprov'].value_counts().head())
print(train['nomprov'].isnull().sum())
print(test['nomprov'].isnull().sum())

train['nomprov'].fillna('MADRID',inplace=True)
test['nomprov'].fillna('MADRID',inplace=True)

le.append('nomprov')

MADRID       173162
BARCELONA     50295
VALENCIA      26182
SEVILLA       24350
CORUÑA, A     16850
Name: nomprov, dtype: int64
MADRID       8921
BARCELONA    2548
VALENCIA     1510
SEVILLA      1391
CORUÑA, A     989
Name: nomprov, dtype: int64
3769
139


In [33]:
# 处理 'ind_actividad_cliente' 字段
print(train['ind_actividad_cliente'].value_counts())
print(test['ind_actividad_cliente'].value_counts())
print(train['ind_actividad_cliente'].isnull().sum())
print(test['ind_actividad_cliente'].isnull().sum())

#填充前一条数据的值
train['ind_actividad_cliente'].fillna(method='pad', inplace=True)

#转换成int型
train['ind_actividad_cliente'] = train['ind_actividad_cliente'].astype('float64')
test['ind_actividad_cliente'] = test['ind_actividad_cliente'].astype('float64')
train['ind_actividad_cliente'] = train['ind_actividad_cliente'].astype('int64')
test['ind_actividad_cliente'] = test['ind_actividad_cliente'].astype('int64')

0.0    288361
1.0    244263
Name: ind_actividad_cliente, dtype: int64
1.0    16439
0.0    12723
Name: ind_actividad_cliente, dtype: int64
1146
0


In [34]:
# 处理 'renta' 字段
print(train.renta.describe())
print(test.renta.describe())

print(train['renta'].isnull().sum())
print(test['renta'].isnull().sum())

#使用地区收入均值填充缺失值
groups = train.groupby(['nomprov'])
temp = groups['renta'].agg([('means','mean')]).reset_index()

#聚合之后还有缺失值，使用中位数填充
temp.fillna(1.021422e+05,inplace=True)
train = train.merge(temp, on='nomprov', how='left')

#用0暂时填充缺失值
train['renta'].fillna(0,inplace=True)
#当收入为0时，用该地区的平均值替代0
for i in range(len(train['renta'])):
    if train['renta'][i] ==0:
        train['renta'][i] = train['means'][i]



count    4.269490e+05
mean     1.366034e+05
std      3.181129e+05
min      6.773130e+03
25%      6.900687e+04
50%      1.021422e+05
75%      1.567044e+05
max      2.889440e+07
Name: renta, dtype: float64
count    2.240500e+04
mean     1.360004e+05
std      3.493648e+05
min      6.773130e+03
25%      6.877887e+04
50%      1.013771e+05
75%      1.551080e+05
max      2.889440e+07
Name: renta, dtype: float64
106821
6757


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [35]:
#使用地区收入均值填充缺失值
import numpy as np
test['renta'].replace('         NA', np.nan, inplace=True) 
test['renta'] = test['renta'].astype('float64')
groups = test.groupby(['nomprov'])
temp = groups['renta'].agg([('means','mean')]).reset_index()

#聚合之后还有缺失值，使用中位数填充
temp.fillna(1.013771e+05,inplace=True)

test = test.merge(temp, on='nomprov', how='left')

test['renta'].fillna(0,inplace=True)

for i in range(len(test['renta'])):
    if test['renta'][i] ==0:
        test['renta'][i] = test['means'][i]

train['renta'] = train['renta'].astype('float64')
test['renta'] = test['renta'].astype('float64')

del train['means']
del test ['means']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [36]:
# 处理 'segmento' 字段
print(train['segmento'].value_counts())
print(test['segmento'].value_counts())
print(train['segmento'].isnull().sum())
print(test['segmento'].isnull().sum())

train['segmento'].fillna('02 - PARTICULARES',inplace=True)
test['segmento'].fillna('02 - PARTICULARES',inplace=True)

le.append('segmento')

02 - PARTICULARES     313806
03 - UNIVERSITARIO    189954
01 - TOP               22228
Name: segmento, dtype: int64
02 - PARTICULARES     16124
03 - UNIVERSITARIO    11579
01 - TOP               1457
Name: segmento, dtype: int64
7782
2


In [37]:
#查看缺失值情况
train.isnull().sum()

ncodpers                 0
ind_empleado             0
pais_residencia          0
sexo                     0
age                      0
ind_nuevo                0
antiguedad               0
indrel                   0
indrel_1mes              0
tiprel_1mes              0
indresi                  0
indext                   0
canal_entrada            0
indfall                  0
cod_prov                 0
nomprov                  0
ind_actividad_cliente    0
renta                    0
segmento                 0
ind_ahor_fin_ult1        0
ind_aval_fin_ult1        0
ind_cco_fin_ult1         0
ind_cder_fin_ult1        0
ind_cno_fin_ult1         0
ind_ctju_fin_ult1        0
ind_ctma_fin_ult1        0
ind_ctop_fin_ult1        0
ind_ctpp_fin_ult1        0
ind_deco_fin_ult1        0
ind_deme_fin_ult1        0
ind_dela_fin_ult1        0
ind_ecue_fin_ult1        0
ind_fond_fin_ult1        0
ind_hip_fin_ult1         0
ind_plan_fin_ult1        0
ind_pres_fin_ult1        0
ind_reca_fin_ult1        0
i

In [38]:
#查看缺失值情况
test.isnull().sum()

ncodpers                 0
ind_empleado             0
pais_residencia          0
sexo                     0
age                      0
ind_nuevo                0
antiguedad               0
indrel                   0
indrel_1mes              0
tiprel_1mes              0
indresi                  0
indext                   0
canal_entrada            0
indfall                  0
cod_prov                 0
nomprov                  0
ind_actividad_cliente    0
renta                    0
segmento                 0
year                     0
month                    0
timestamp                0
fyear                    0
fmonth                   0
fday                     0
ftimestamp               0
dtype: int64

In [39]:
# 分类特征LabelEncoder
from sklearn.preprocessing import LabelEncoder

# 对于分类特征进行特征值编码
for feature in le:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])

## 模型训练

In [40]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [41]:
train_label = pd.DataFrame()
test_label = pd.DataFrame()

#将label和训练集分离
for column in targets:
    train_label[column] = train[column]
    del train[column]

#构建一个test副本存放预测值
for column in test.columns:
    test_label[column] = test[column]

In [42]:
#对于24个标签分别进行预测 1000 和 50 早停太慢，改成100和50早停
for target in targets:
    X_train, X_valid, y_train, y_valid = train_test_split(train, train_label[target], test_size=0.3, random_state=42)
    
    # 创建模型，训练模型
    gbm = lgb.LGBMClassifier(silent=True,num_leaves=51,max_depth=10,boosting_type='gbdt',objective='binary',learning_rate=0.015,n_estimators=100,
                         subsample=0.75,subsample_freq=2,reg_lambda=0.28,reg_alpha=0.12,colsample_bytree=0.8,min_child_samples=300,min_split_gain=0.1)
    
    gbm.fit(X_train, y_train,eval_set=[(X_train, y_train), (X_valid, y_valid)],eval_metric='auc', early_stopping_rounds=50)
    # 预测数据集
    prob = gbm.predict_proba(test)
    test_label[target] = pd.Series(prob[:,1])

[1]	training's auc: 0.998904	training's binary_logloss: 0.00083657	valid_1's auc: 0.998726	valid_1's binary_logloss: 0.000893937
Training until validation scores don't improve for 50 rounds.
[2]	training's auc: 0.998963	training's binary_logloss: 0.000764816	valid_1's auc: 0.998347	valid_1's binary_logloss: 0.000828977
[3]	training's auc: 0.999441	training's binary_logloss: 0.000731701	valid_1's auc: 0.999095	valid_1's binary_logloss: 0.00079812
[4]	training's auc: 0.999453	training's binary_logloss: 0.000684272	valid_1's auc: 0.999082	valid_1's binary_logloss: 0.00075419
[5]	training's auc: 0.999817	training's binary_logloss: 0.000623224	valid_1's auc: 0.999665	valid_1's binary_logloss: 0.000697111
[6]	training's auc: 0.99987	training's binary_logloss: 0.000588393	valid_1's auc: 0.999773	valid_1's binary_logloss: 0.000659307
[7]	training's auc: 0.999926	training's binary_logloss: 0.000558453	valid_1's auc: 0.999866	valid_1's binary_logloss: 0.000629547
[8]	training's auc: 0.99997	trai

In [43]:
#提取预测的概率值
templabel = test_label[targets]

#使用index存放候选集
index = pd.DataFrame()
index['ncodpers'] = test_label['ncodpers']

In [44]:
#概率第一名
temp =  templabel.idxmax(axis=1)
index ['one'] = temp

count = 0
for i in temp:
    templabel[i][count] = 0
    count+=1

#概率第二名
temp =  templabel.idxmax(axis=1)
index['two'] = temp

count = 0
for i in temp:
    templabel[i][count] = 0
    count+=1

#概率第3名
temp =  templabel.idxmax(axis=1)
index['three'] = temp

count = 0
for i in temp:
    templabel[i][count] = 0
    count+=1

#概率第4名
temp =  templabel.idxmax(axis=1)
index['fourth'] = temp

count = 0
for i in temp:
    templabel[i][count] = 0
    count+=1
    
#概率第5名
temp =  templabel.idxmax(axis=1)
index['five'] = temp

count = 0
for i in temp:
    templabel[i][count] = 0
    count+=1

#概率第6名
temp =  templabel.idxmax(axis=1)
index['six'] = temp

count = 0
for i in temp:
    templabel[i][count] = 0
    count+=1
    
#概率第7名
temp =  templabel.idxmax(axis=1)
index['seven'] = temp

count = 0
for i in temp:
    templabel[i][count] = 0
    count+=1

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [45]:
index['added_products'] = index['one']+' '+index['two']+" "+index['three']+" "+index['fourth']+' '+index['five']+" "+index['six']+" "+index['seven']

#删除多余列
temp = index.columns.drop(['ncodpers','added_products'])

for i in temp:
    del index[i]
    
submission = pd.read_csv('sub.csv')

submission['added_products'] = pd.Series(index['added_products'])

submission.to_csv('sub.csv', index=False)

In [48]:
submission

,ncodpers,added_products
0,657788,ind_cco_fin_ult1 ind_recibo_ult1 ind_ctop_fin_...
1,657794,ind_cco_fin_ult1 ind_dela_fin_ult1 ind_ecue_fi...
2,657770,ind_cco_fin_ult1 ind_recibo_ult1 ind_ctop_fin_...
3,658056,ind_cco_fin_ult1 ind_reca_fin_ult1 ind_recibo_...
4,657869,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
...,...,...
29157,1166847,ind_cco_fin_ult1 ind_recibo_ult1 ind_cno_fin_u...
29158,1166605,ind_cco_fin_ult1 ind_recibo_ult1 ind_ctop_fin_...
29159,1166829,ind_cco_fin_ult1 ind_recibo_ult1 ind_cno_fin_u...
29160,1166827,ind_cco_fin_ult1 ind_recibo_ult1 ind_ctop_fin_...


In [47]:
submission['added_products'][0]

'ind_cco_fin_ult1 ind_recibo_ult1 ind_ctop_fin_ult1 ind_cno_fin_ult1 ind_reca_fin_ult1 ind_nom_pens_ult1 ind_nomina_ult1'